In [1]:
# ============================================================
# CÉLULA 1: Importação das Bibliotecas e Carregamento dos Dados
# ============================================================
import pandas as pd
import numpy as np
import joblib
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import warnings

warnings.filterwarnings("ignore")
print("Bibliotecas carregadas.")

# --- Importar dados preparados ---
DATA_PATH = "../data/model_inputs/model_input.csv"
try:
    df = pd.read_csv(DATA_PATH)
    print(f"Dataset carregado com sucesso: {df.shape[0]} linhas, {df.shape[1]} colunas.")
    print("   Amostra dos dados:")
    display(df.head())
except FileNotFoundError:
    raise RuntimeError(f"ERRO: O arquivo '{DATA_PATH}' não foi encontrado. Execute o notebook 'data_preparation' primeiro.")

Bibliotecas carregadas.
Dataset carregado com sucesso: 13 linhas, 24 colunas.
   Amostra dos dados:


,poi_nome,provincia,latitude,longitude,tempmedanual,precipitacaoanual,ndvi,evi,ndwi,lulccodigo,...,idh,taxaurbanizacao,empregoturismo,distanciaestradaprincipalkm,distanciacidadekm,acessibilidade,tipoviaacesso,infraestrutura,servicosdisponiveis,densidadepopulacionalhabkm
0,Quedas de Calandula,Malanje,-9.074067,16.000316,26.3,1290.0,0.72,0.64,0.09,20.0,...,0.522,80.6,3.2,0.0,0.0,0,0,0,0,0.0
1,Miradouro da Lua,Luanda,-9.196087,13.074263,27.8,320.0,0.26,0.18,0.12,60.0,...,0.655,68.6,8.1,2.5,45.0,Boa,Pavimentada,Completa,"Energia, telecom",430.0
2,Museu Kulumbimbi,Zaire,-6.271675,14.250108,25.8,1450.0,0.78,0.71,0.12,20.0,...,0.633,44.6,7.7,0.0,0.0,0,0,0,0,0.0
3,Reserva Parcial do Namibe,Namibe,-15.766645,12.399980,0.0,0.0,0.00,0.00,0.00,0.0,...,0.700,59.2,6.5,0.0,0.0,0,0,0,0,0.0
4,Fortaleza de São Miguel,Luanda,-8.785376,13.234109,27.9,325.0,0.15,0.12,0.11,50.0,...,0.502,56.3,3.2,0.0,0.0,0,0,0,0,0.0


In [2]:
# ===================================================================
# CÉLULA 2: Definição das Variáveis e Pipeline de Pré-processamento
# ===================================================================

# --- 1. Separar o alvo (target) das features ---
# Garantir que a coluna 'idh' existe antes de prosseguir
if 'idh' not in df.columns:
    raise ValueError("A coluna alvo 'idh' não foi encontrada no DataFrame.")
    
X = df.drop(columns=['idh'])
y = df['idh']

# --- 2. Seleção Estratégica de Features ---
# Remover colunas que são apenas identificadores e não devem ser usadas para treinar o modelo.
# 'poi_nome' tem valores únicos e não ajuda a generalizar.
cols_to_drop_from_features = ['poi_nome']
X = X.drop(columns=cols_to_drop_from_features, errors='ignore')
print(f"Colunas identificadoras removidas das features: {cols_to_drop_from_features}")


# --- 3. Identificar os tipos de features restantes ---
numeric_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

# A 'provincia' é a única feature categórica que queremos tratar.
# Outras colunas de objeto, se existirem, devem ser investigadas.
if 'provincia' in categorical_features:
     print("'provincia' será tratada como feature categórica.")
else:
     print("Nenhuma feature categórica de interesse ('provincia') encontrada.")


print(f"\n Features Numéricas para escalonar ({len(numeric_features)}): {numeric_features[:5]}...") # Mostra apenas as 5 primeiras
print(f"Features Categóricas para OneHotEncode ({len(categorical_features)}): {categorical_features}")

# --- 4. Criar o pré-processador com ColumnTransformer ---
# Define os passos de transformação para cada tipo de coluna.
preprocessor = ColumnTransformer(
    transformers=[
        # Pipeline para features numéricas: aplicar escalonamento padrão
        ('num', StandardScaler(), numeric_features),
        
        # Pipeline para features categóricas: aplicar One-Hot Encoding
        # 'handle_unknown='ignore'' é crucial para evitar erros se uma nova categoria aparecer na predição
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough' # Mantém colunas não especificadas (se houver), embora não deva haver.
)

print("\n Pré-processador definido com sucesso.")

Colunas identificadoras removidas das features: ['poi_nome']
'provincia' será tratada como feature categórica.

 Features Numéricas para escalonar (17): ['latitude', 'longitude', 'tempmedanual', 'precipitacaoanual', 'ndvi']...
Features Categóricas para OneHotEncode (5): ['provincia', 'acessibilidade', 'tipoviaacesso', 'infraestrutura', 'servicosdisponiveis']

 Pré-processador definido com sucesso.


In [3]:
# Treinar modelos
# ============================================================
#  Treinamento e Avaliação do Modelo
# ============================================================

# --- Dividir os dados em conjuntos de treino e teste ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Dados divididos em {len(X_train)} amostras de treino e {len(X_test)} de teste.\n")

# --- Criar o Pipeline final que une o pré-processador e o modelo ---
# O Pipeline garante que os mesmos passos de pré-processamento sejam aplicados
# de forma consistente durante o treino e a predição.
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, random_state=42))
])

# --- Treinar o modelo (o pipeline inteiro) ---
pipeline.fit(X_train, y_train)
print(" Modelo treinado com sucesso!")

# --- Avaliar o modelo no conjunto de teste ---
y_pred = pipeline.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"\n--- Desempenho do Modelo ---")
print(f" R² (Coeficiente de Determinação): {r2:.3f}")
print(f" RMSE (Raiz do Erro Quadrático Médio): {rmse:.3f}")

# Visualização: comparação real vs predito
plt.figure(figsize=(7, 5))
sns.scatterplot(x=y_test, y=y_pred)
plt.xlabel("Valor Real (IDH)")
plt.ylabel("Valor Predito")
plt.title("Real vs Predito — Modelo Random Forest")
plt.grid(True, alpha=0.3)
plt.show()

Dados divididos em 10 amostras de treino e 3 de teste.

 Modelo treinado com sucesso!

--- Desempenho do Modelo ---
 R² (Coeficiente de Determinação): -2.496
 RMSE (Raiz do Erro Quadrático Médio): 0.096


In [4]:
#  Análise exploratória dos dados
# ============================================================

print(" Informações gerais:")
df.info()

print("\n Estatísticas descritivas:")
display(df.describe())

# Verificar valores ausentes
print("\n Valores ausentes:")
display(df.isnull().sum())

# Visualização rápida da distribuição das variáveis
df.hist(figsize=(12, 8), bins=15, color="#006699")
plt.suptitle("Distribuição das Variáveis Numéricas", fontsize=14)
plt.show()

 Informações gerais:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 24 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   poi_nome                     13 non-null     object 
 1   provincia                    13 non-null     object 
 2   latitude                     13 non-null     float64
 3   longitude                    13 non-null     float64
 4   tempmedanual                 13 non-null     float64
 5   precipitacaoanual            13 non-null     float64
 6   ndvi                         13 non-null     float64
 7   evi                          13 non-null     float64
 8   ndwi                         13 non-null     float64
 9   lulccodigo                   13 non-null     float64
 10  altitude                     13 non-null     float64
 11  populacao                    13 non-null     float64
 12  densidadepop                 13 non-null     float64
 13  p

,latitude,longitude,tempmedanual,precipitacaoanual,ndvi,evi,ndwi,lulccodigo,altitude,populacao,densidadepop,pibpercapita,idh,taxaurbanizacao,empregoturismo,distanciaestradaprincipalkm,distanciacidadekm,densidadepopulacionalhabkm
count,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000
mean,-11.228358,14.762060,22.492308,841.538462,0.463077,0.406923,0.094615,33.076923,840.153846,131851.846154,163.969231,2244.288462,0.540538,55.061538,7.592308,0.607692,7.115385,118.076923
std,3.192837,2.906671,7.427703,524.723676,0.304915,0.270166,0.060638,18.878831,797.184718,52185.903326,164.322348,1151.451176,0.179059,22.983456,4.476692,0.900427,12.822756,187.010901
min,-15.766645,12.399980,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-14.816045,13.195976,20.500000,320.000000,0.150000,0.120000,0.040000,20.000000,45.000000,118268.000000,25.000000,1471.820000,0.512000,42.800000,4.100000,0.000000,0.000000,0.000000
50%,-11.020670,13.380655,24.500000,980.000000,0.580000,0.490000,0.100000,30.000000,850.000000,129958.000000,77.100000,2347.390000,0.569000,59.200000,7.700000,0.000000,0.000000,0.000000
75%,-8.942211,14.673313,27.100000,1250.000000,0.720000,0.640000,0.120000,50.000000,1150.000000,154867.000000,298.100000,2889.860000,0.633000,68.600000,10.800000,1.000000,10.000000,125.000000
max,-6.271675,21.695327,28.100000,1550.000000,0.800000,0.720000,0.200000,60.000000,2200.000000,221458.000000,438.600000,4149.690000,0.703000,82.900000,14.100000,2.500000,45.000000,520.000000



 Valores ausentes:


poi_nome                       0
provincia                      0
latitude                       0
longitude                      0
tempmedanual                   0
precipitacaoanual              0
ndvi                           0
evi                            0
ndwi                           0
lulccodigo                     0
altitude                       0
populacao                      0
densidadepop                   0
pibpercapita                   0
idh                            0
taxaurbanizacao                0
empregoturismo                 0
distanciaestradaprincipalkm    0
distanciacidadekm              0
acessibilidade                 0
tipoviaacesso                  0
infraestrutura                 0
servicosdisponiveis            0
densidadepopulacionalhabkm     0
dtype: int64

In [5]:
# Avaliar variáveis mais importantes


# Extrair os nomes das features DEPOIS da transformação do pré-processador
feature_names_out = pipeline.named_steps['preprocessor'].get_feature_names_out()

# Extrair as importâncias do modelo treinado dentro do pipeline
importances = pipeline.named_steps['model'].feature_importances_

# Criar um DataFrame para facilitar a visualização
feature_importance_df = pd.DataFrame({
    'feature': feature_names_out,
    'importance': importances
}).sort_values('importance', ascending=False)

# --- Gerar o gráfico ---
plt.figure(figsize=(10, 8))
sns.barplot(x='importance', y='feature', data=feature_importance_df.head(20), palette='viridis')
plt.title("Top 20 Variáveis Mais Importantes")
plt.xlabel("Importância Relativa")
plt.ylabel("Variável")
plt.tight_layout()
plt.show()

In [6]:
# Salvar modelo
# ============================================================
#  Salvamento do Modelo (Pipeline Completo)
# ============================================================

# Definir o caminho e o nome do arquivo do modelo
MODEL_DIR = "../data/model_inputs"
MODEL_PATH = os.path.join(MODEL_DIR, "tourism_model.pkl")

# Criar o diretório se ele não existir
os.makedirs(MODEL_DIR, exist_ok=True)

# Salvar o objeto 'pipeline' inteiro. Ele contém tudo o que é necessário
# para fazer novas previsões: o pré-processador e o modelo treinado.
joblib.dump(pipeline, MODEL_PATH)

print(f" Pipeline completo salvo com sucesso em: {MODEL_PATH}")

 Pipeline completo salvo com sucesso em: ../data/model_inputs\tourism_model.pkl


In [7]:
# Entrega esperada: models/tourism_model.pkl
# ============================================================
# Verificação da Entrega
# ============================================================

# Este bloco apenas confirma que o arquivo foi criado no local esperado.
if os.path.exists(MODEL_PATH):
    print(f"\n Entrega confirmada: o arquivo '{os.path.basename(MODEL_PATH)}' existe no diretório '{MODEL_DIR}'.")
else:
    print(f"\n Atenção: o arquivo do modelo não foi encontrado no local esperado.")


 Entrega confirmada: o arquivo 'tourism_model.pkl' existe no diretório '../data/model_inputs'.
